In [1]:
# Initial Imports
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
from datetime import datetime

import finnhub
import ta #Technical Analysis library

load_dotenv()

True

In [2]:
# Setting up Finnhub API
finnhub_api_key = os.getenv("FINNHUB_API_KEY")
finnhub_sandbox_key = os.getenv("FINNHUB_SANDBOX_KEY")
finnhub_client = finnhub.Client(api_key = finnhub_api_key)

In [3]:
def getUNIX(date):
    """
    Input date in YYYY-MM-DD format (as a string) and returns the associated UNIX timestamp
    """
    # Parsing the input date
    dateparts = date.split("-")
    year = int(dateparts[0])
    month = int(dateparts[1])
    day = int(dateparts[2])
    
    unix = int((datetime(year, month, day) - datetime(1970,1,1)).total_seconds())
    return unix

In [4]:
def getYMD(unix):
    """
    Input a UNIX timestamp and returns a date in the format of YYYYMMDD - H
    Any additional minutes or seconds are dropped
    """
    ts = int(unix)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d-%H')

In [5]:
def getOHLCV(ticker, startDate, endDate, grain = "D"):
    """
    Input a ticker, startDate (YYYY-MM-DD), endDate (YYYY-MM-DD)
    Returns daily open, high, low, close, and volume (in that order) in a pandas dataframe for the given criteria
    """

    # Converting to UNIX timestamp
    startDate = getUNIX(startDate)
    endDate = getUNIX(endDate)
    
    # Calling Finnhub API for candles data
    if grain == "H":
        candlesData = finnhub_client.stock_candles(ticker, '60', startDate, endDate)
        OHLCV = pd.DataFrame(candlesData)
    else:
        candlesData = finnhub_client.stock_candles(ticker, 'D', startDate, endDate)
        OHLCV = pd.DataFrame(candlesData)
    
    # Dropping the column denoting status of response and any null fields
    OHLCV.drop(columns = "s", inplace = True)
    OHLCV.dropna(inplace = True)
    
    # Renaming columns for ease of interpretation
    OHLCV = OHLCV.rename(columns = {
        "c":"close",
        "h":"high",
        "l":"low",
        "o":"open",
        "t":"date",
        "v":"volume"
    })
    
    # Converting UNIX timestamp to date and setting date as index
    OHLCV["date"] = OHLCV["date"].apply(getYMD)
    OHLCV.set_index(OHLCV['date'], inplace = True)
    OHLCV.drop(columns = "date", inplace = True)
    OHLCV.index = pd.to_datetime(OHLCV.index)
    OHLCV.sort_index(inplace = True)
    
    # Reordering columns to match OHLCV
    OHLCV = OHLCV[["open", "high", "low", "close", "volume"]]
    
    return OHLCV

In [6]:
def getTechIndicators(OHLCV, dropnull = True):
    """
    Input OHLCV dataframe. Please note that due to the windows of the SMAs, at least 200 rows worth of data will be required. 
    Rows with null values will be automatically dropped
    
    Calculates daily returns, previous close, and lagged returns, and the following technical indicators:
    
    Simple Moving Average of Closing prices with 20, 50, and 100 row windows
    Exponential Moving Average of Daily Returns Volatility with 20, 50, and 100 row windows
    Moving Average Convergence Divergence (MACD)

    On Balance Volume (OBV)
    Chaikin Money Flow (CMF)
    
    Awesome Oscillator (AOsc)
    
    Relative Strength Index (RSI)
    Stochastic Oscillator (SOsc)
    """
    TechInd = OHLCV.copy()
    
    # Calculating Returns
    TechInd["DR"] = TechInd["close"].pct_change()
    
    # Previous Close
    TechInd["prevClose"] = TechInd["close"].shift()
    
    # Lagged Returns
    TechInd["laggedReturns"] = TechInd["DR"].shift()
    
    # Calculating the Simple Moving Averages of Closing Prices
    TechInd["SMA20"] = TechInd["close"].rolling(window = 20).mean()
    TechInd["SMA50"] = TechInd["close"].rolling(window = 50).mean()
    TechInd["SMA100"] = TechInd["close"].rolling(window = 100).mean()
    
    # Calculating EWM of Daily Return Volatility
    TechInd["EWM20V"] = TechInd["DR"].ewm(halflife = 20).std()
    TechInd["EWM50V"] = TechInd["DR"].ewm(halflife = 50).std()
    TechInd["EWM100V"] = TechInd["DR"].ewm(halflife = 100).std()
    
    # Calculating MACD
    ewm26 = TechInd['close'].ewm(halflife = 26).mean()
    ewm12 = TechInd['close'].ewm(halflife = 12).mean()
    TechInd["MACD"] = ewm12 - ewm26
    
    TechInd["MACDEWM9"] = TechInd["MACD"].rolling(window = 9).mean()
    
    # Calculating on balance volume
    TechInd["OBV"] = ta.volume.on_balance_volume(TechInd["close"], TechInd["volume"])
    
    # Calculating Chaikin Money Flow
    TechInd["CMF"] = ta.volume.ChaikinMoneyFlowIndicator(
        high = TechInd["high"], 
        low = TechInd["low"], 
        close = TechInd["close"], 
        volume = TechInd["volume"]).chaikin_money_flow()
    
    # Calculating Awesome Oscillator
    TechInd["AOsc"] = ta.momentum.AwesomeOscillatorIndicator(
        high = TechInd["high"], 
        low = TechInd["low"]).awesome_oscillator()
    
    # Calculating RSI
    TechInd["RSI"] = ta.momentum.RSIIndicator(close = TechInd["close"]).rsi()
    
    # Calculating Stochastic Oscillator
    TechInd["SOsc"] = ta.momentum.StochasticOscillator(
        high = TechInd["high"], 
        low = TechInd["low"], 
        close = TechInd["close"]).stoch()
    
    # Dropping rows with null values
    if dropnull:
        TechInd.dropna(inplace = True)
    
    return TechInd

In [7]:
def indAnalysis(TechInd):
    """
    Gives Bullish (1.0), Bearish (-1.0), or Neutral (0.0) outlook based on provided technical indicators.
    
    TechInd dataframe should contain the following indicators:
        - Simple Moving Average if Closing prices with 20, 50, and 100 row windows
        - Exponential Moving Average of Daily Returns Volatility with 20, 50, and 100 row windows
        - Moving Average Convergence Divergence (MACD)
        On Balance Volume (OBV)
        - Chaikin Money Flow (CMF)
        - Awesome Oscillator (AOsc)
        Relative Strength Index (RSI)
        Stochastic Oscillator (SOsc)
    """
    TechA = TechInd.iloc[:,0:5]
    
    # Signal for SMA crossovers
    SMA2050Bull = np.where(TechInd["SMA20"] > TechInd["SMA50"], 1.0, 0.0)
    SMA2050Bear = np.where(TechInd["SMA20"] < TechInd["SMA50"], -1.0, 0.0)
    TechA["SMA20-50"] = SMA2050Bull + SMA2050Bear
    
    SMA50100Bull = np.where(TechInd["SMA50"] > TechInd["SMA100"], 1.0, 0.0)
    SMA50100Bear = np.where(TechInd["SMA50"] < TechInd["SMA100"], -1.0, 0.0)
    TechA["SMA50-100"] = SMA50100Bull + SMA50100Bear
    
    # Signal for EWM Volatility crossovers
    EWM2050VBull = np.where(TechInd["EWM20V"] < TechInd["EWM50V"], 1.0, 0.0)
    EWM2050VBear = np.where(TechInd["EWM20V"] > TechInd["EWM50V"], -1.0, 0.0)
    TechA["EWM20-50V"] = EWM2050VBull + EWM2050VBear
    
    EWM50100VBull = np.where(TechInd["EWM50V"] < TechInd["EWM100V"], 1.0, 0.0)
    EWM50100VBear = np.where(TechInd["EWM50V"] > TechInd["EWM100V"], -1.0, 0.0)
    TechA["EWM50-100V"] = EWM50100VBull + EWM50100VBear
    
    # Signal for MACD crossover with MACD signal line
    MACDBull = np.where(TechInd["MACD"] > TechInd["MACDEWM9"], 1.0, 0.0)
    MACDBear = np.where(TechInd["MACD"] < TechInd["MACDEWM9"], -1.0, 0.0)
    TechA["MACDCross"] = MACDBear + MACDBull
    
    # Signal for CMF
    CMFBull = np.where(TechInd["CMF"] > 0, 1.0, 0.0)
    CMFBear = np.where(TechInd["CMF"] < 0, -1.0, 0.0)
    TechA["CMFSig"] = CMFBull + CMFBear
    
    # Signal for AOsc
    AOscBull = np.where(TechInd["AOsc"] > 0, 1.0, 0.0)
    AOscBear = np.where(TechInd["AOsc"] < 0, -1.0, 0.0)
    TechA["AOscSig"] = AOscBull + AOscBear
    
    # Signal for RSI
    RSIBull = np.where(TechInd["RSI"] <= 30, 1.0, 0.0)
    RSIBear = np.where(TechInd["RSI"] >= 70, -1.0, 0.0)
    TechA["RSISig"] = RSIBull + RSIBear
    
    # Signal for SOsc
    SOscBull = np.where(TechInd["SOsc"] <= 20, 1.0, 0.0)
    SOscBear = np.where(TechInd["SOsc"] >= 80, -1.0, 0.0)
    TechA["SOscSig"] = SOscBull + SOscBear
    
    return TechA

In [8]:
def getLastSig(TechSig):
    """
    Pulls the last row of the Indicator signals and reshapes the dataframe in a format for hvplot tables
    """
    lastRow = TechSig.tail(1)
    lastRow.drop(columns = ["open", "high", "low", "close", "volume"], inplace = True)
    
    # Renaming for readability
    lastRow = lastRow.rename(columns = {
        "SMA20-50" : "20-50 Period Closing SMA Crossover",
        "SMA50-100" : "50-100 Period Closing SMA Crossover",
        "EWM20-50V" : "20-50 Period Volatility EWM Crossover",
        "EWM50-100V" : "50-100 Period Volatility EWM Crossover",
        "MACDCross" : "Moving Average Convergence Divergence Crossover",
        "CMFSig" : "Chaikin Money Flow",
        "AOscSig" : "Awesome Oscillator",
        "RSISig" : "Relative Strength Index",
        "SOscSig" : "Stochastic Oscillator"
    })
    
    signalTabledf = lastRow.transpose()
    signalTabledf.columns = ["Outlook"]
    signalTabledf["Outlook"] = signalTabledf["Outlook"].apply(str)
    signalTabledf["Outlook"] = signalTabledf["Outlook"].str.replace("-1.0", "Bearish")
    signalTabledf["Outlook"] = signalTabledf["Outlook"].str.replace("1.0", "Bullish")
    signalTabledf["Outlook"] = signalTabledf["Outlook"].str.replace("0.0", "Neutral")
    signalTabledf["Measure"] = signalTabledf.index
    signalTabledf = signalTabledf[["Measure", "Outlook"]]
    
    return signalTabledf

In [9]:
def getLastOHLCV(TechSig):
    """
    Pull the last row of the Indicator signals for OHLCV and reshapes the dataframe in a format for hvplot tables
    """
    lastRow = TechSig.tail(1)
    lastRow = lastRow[["open", "high", "low", "close", "volume"]]
    
    ohlcvTabledf = lastRow.transpose()
    ohlcvTabledf = ohlcvTabledf.round(2)
    ohlcvTabledf.columns = ["Value"]
    ohlcvTabledf["Measure"] = ohlcvTabledf.index 
    ohlcvTabledf = ohlcvTabledf[["Measure", "Value"]]

    return ohlcvTabledf

In [21]:
def getSignalRec(ticker, startDate = "2020-06-01", endDate = "2021-01-15"):
    """
    Pull the last row of the Indicator signals and gives a recommendation to buy, sell, or hold based on the signals. 
    If the bullish outlook outweighs the bearish outlook by 2, the signal will be to buy.
    If the bearish outlook outweighs the bearish outlook by 2, the signal will be to sell.
    Otherwise, the signal will be to hold
    Buy = 1.0
    Sell = -1.0
    Hold = 0.0
    """
    rec = 0.0
    
    # Calculating Technical Indicators
    tickerInfo = getOHLCV(ticker, startDate, endDate, grain = "D")
    tickerInd = getTechIndicators(tickerInfo)
    
    lastRow = tickerInd.tail(1)
    lastRow.drop(columns = ["open", "high", "low", "close", "volume"], inplace = True)
    
    signalTabledf = lastRow.transpose()
    signalTabledf.columns = ["Outlook"]
    
    recSig = signalTabledf["Outlook"].sum()
    
    if recSig >= 2:
        rec = 1.0
    elif recSig <= -2:
        rec = -1.0
    
    return rec